## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
# sys.path

In [ ]:
import pandas as pd

In [ ]:
from market_oracle_lib import hello
from market_oracle_lib.data import t_bank, yfinance, apimoex
from market_oracle_lib.data.data_funcs import (
    create_data_loaders,
    create_data_frames,
    create_data_sets,
    create_base_data_frames,
)
from market_oracle_lib.consts import RU_DEFAULT_SYMBOLS

In [ ]:
with open("../.secret_env", "r") as f:
    T_TOKEN = f.read().strip().split("=")[1]
print(T_TOKEN)

In [ ]:
hello()

## Try to get data

In [ ]:
# train_dl, val_dl, test_dl = create_data_loaders(yfinance.get_symbol_data, interval="1m")

In [ ]:
# train_dl, val_dl, test_dl = create_data_frames(yfinance.get_symbol_data, interval="1d", period="max")
# train_dl
# # next(iter(train_dl))[0].shape, next(iter(train_dl))[1].shape

In [ ]:
train_dl, val_dl, test_dl = create_data_frames(apimoex.get_symbol_data,
                                               interval="1d",
                                               symbols=RU_DEFAULT_SYMBOLS)

In [ ]:
train_df, val_df, test_df = create_data_frames(t_bank.get_symbol_data, interval="1d",
                                               token=T_TOKEN,
                                            #    start_date="2024-01-01",
                                            #    end_date="2024-09-02",
                                               symbols=RU_DEFAULT_SYMBOLS)
# train_dl
# next(iter(train_dl))[0].shape, next(iter(train_dl))[1].shape

In [ ]:
train_df

In [ ]:
train_dl.iloc[0].values

In [ ]:
train_dl.head()

In [ ]:
apimoex.get_symbol_data(RU_DEFAULT_SYMBOLS[0], interval="1d").sort_values(by="Date", ascending=False)

In [ ]:
t_bank.get_symbol_data(symbol=RU_DEFAULT_SYMBOLS[0], token=T_TOKEN,
                        # start_date="2024-01-01", end_date="2024-09-02",
                        interval="1d").sort_values(by="Date", ascending=False)

In [ ]:
from tinkoff.invest import Client
with Client(T_TOKEN) as client:
    print(*client.instruments.shares().instruments, sep="\n")

In [ ]:
# YNDX, POLY
instrument_info = t_bank.find_instrument_by_ticker("SBER", T_TOKEN)
instrument_info

In [ ]:
# Получаем данные
df = t_bank.get_symbol_data("SBER", token=T_TOKEN)
df

In [ ]:
from datetime import datetime, timedelta, timezone
from tinkoff.invest import Client, CandleInterval
from tinkoff.invest.schemas import CandleSource
from tinkoff.invest.utils import quotation_to_decimal

with Client(T_TOKEN) as client:
    # Получаем исторические данные по FIGI
    data = []
    for candle in client.get_all_candles(
        instrument_id=instrument_info["figi"],
        from_=datetime.now(timezone.utc) - timedelta(days=365),
        interval=CandleInterval.CANDLE_INTERVAL_HOUR,
        candle_source_type=CandleSource.CANDLE_SOURCE_UNSPECIFIED,
    ):
        data.append({
            'Date': candle.time,
            'Open': float(quotation_to_decimal(candle.open)),
            'High': float(quotation_to_decimal(candle.high)),
            'Low': float(quotation_to_decimal(candle.low)),
            'Close': float(quotation_to_decimal(candle.close)),
            'Volume': candle.volume,
        })

In [ ]:
pd.DataFrame(data)

In [ ]:
# Пример получения технических индикаторов
pd.DataFrame(t_bank.get_indicators(instrument_info, T_TOKEN))

In [ ]:
from tinkoff.invest.services import MarketDataService
from datetime import datetime, timedelta, timezone
from tinkoff.invest.schemas import CandleSource, GetTechAnalysisRequest, IndicatorType, IndicatorInterval, TypeOfPrice, Deviation, Smoothing
from tinkoff.invest import Client, CandleInterval

with Client(T_TOKEN) as client:
    market_data_service = client.market_data
    req = GetTechAnalysisRequest(
        indicator_type=IndicatorType.INDICATOR_TYPE_SMA,
        instrument_uid=instrument_info["instrument"].uid,
        from_=datetime.now(timezone.utc) - timedelta(days=365),
        to=datetime.now(timezone.utc),
        interval=IndicatorInterval.INDICATOR_INTERVAL_ONE_DAY,  # Используем дневной интервал
        type_of_price=TypeOfPrice.TYPE_OF_PRICE_CLOSE,
        length=20,
    )
    res = market_data_service.get_tech_analysis(request=req)

    print(*res.technical_indicators, sep="\n")

In [ ]:
train_dl.tail()

In [ ]:
print(*dir(client), sep="\n")

In [ ]:
train_dl, val_dl, test_dl

data_dir = os.path.join(os.getcwd(), "../saved_data/")
train_dl.to_csv(data_dir + "big_train_dl.csv")
# val_dl.to_csv(data_dir + "val_dl3.csv")
# test_dl.to_csv(data_dir + "test_dl3.csv")
# os.getcwd()

In [ ]:
train_dl.head()

## Train

In [ ]:
import pytorch_lightning as pl
from market_oracle_lib.model import LSTMModel

# Создаем модель
model = LSTMModel(
    input_size=23,  # Размерность входных данных (количество признаков)
    hidden_size=64,
    num_layers=2,
    output_size=1,  # Предсказываем одно значение
    learning_rate=0.001
)

# Создаем тренер
trainer = pl.Trainer(
    max_epochs=5,
    accelerator='auto',  # Автоматически определяет доступное устройство (CPU/GPU)
    devices=1,
    enable_progress_bar=True,
    enable_model_summary=True,
)

# Обучаем модель
trainer.fit(model, train_dl, val_dl)

# Оцениваем на тестовых данных
test_results = trainer.test(model, test_dl)
print(f"Test results: {test_results}")

## Validate

In [ ]:
from market_oracle_lib.validate import validate_and_visualize

validate_and_visualize(model, test_dl)